In [1]:
import emapy as epy
import sys;
import pandas as pd

In [2]:
barris = epy.getDatabase(
    'barris',
    'json',
    '../data/raw/barris.geojson',
    '',
    True,
    0,
    1,
    'cartodb_id')  

In [3]:
csvFile = '../data/external/densidadPoblacion/superficie-dens2015.csv'
csvData = epy.getDataOfCsv(
                csvFile, 
                ";"
)
dataSize = len(csvData)
barrisSize = len(barris[1])
allBarris = barris[1]

In [4]:
for index, row in csvData.iterrows():
    csvData.set_value(
        index, 
        'Barris', 
        epy.removeIntInxString(
            csvData.get_value(index, 'Barris'), 
            '.').lower())

In [5]:
df = pd.DataFrame({'id' : [], 'data': []})
allId = dict()

for x in range(barrisSize):
    key = allBarris[x]['geometry']['properties']['cartodb_id']
    name = allBarris[x]['geometry']['properties']['neighbourhood']
    for index, row in csvData.iterrows():
        if csvData.iloc[index][1].strip().lower() == name.strip().lower():
            csvNameData = csvData.iloc[index][1]    
            if key in allId:
                allId[key] = -1
            else:
                num = csvData.iloc[index][u'Població']
                if epy.is_number(num) == False:                   
                    num = num.replace('.', '')
                    num = float(num.replace(',', '.'))
                else:
                    num = str(num).replace('.', '')
                    num = float(num.replace(',', '.'))
                allId[key] = num
                
for x in range(barrisSize):
    key = allBarris[x]['geometry']['properties']['cartodb_id']
    if key in allId:
        row = [key, allId[key] * 1.0]  
        df.loc[len(df), ['id', 'data']] = row  
    else:
        df.loc[len(df), ['id', 'data']] = [key,0]

In [6]:
map = epy.mapCreation(41.388790,2.158990)
epy.mapChoropleth(map, 
                 '../data/raw/barris.geojson',
                 'feature.properties.cartodb_id',
                 df,
                 'id',
                 'data',
                 'YlGn',
                 0.7,
                 0.3,
                 [0, 10000, 20000, 37000, 50000, 60000],
                 'bars / barri')


In [7]:
locationAreaBoundingBox = (41.3248770036,2.0520401001,41.4829908452,2.2813796997)
allBars = epy.getDatabaseFromOSM(
    'bares', 
    'amenity',
    False,
    True, 
    locationAreaBoundingBox, 
    'bar')

allRestaurants = epy.getDatabaseFromOSM(
    'restaurants', 
    'amenity',
    False,
    True, 
    locationAreaBoundingBox, 
    'restaurant')

allFastFood = epy.getDatabaseFromOSM(
    'fast food', 
    'amenity',
    False,
    True, 
    locationAreaBoundingBox, 
    'fast_food')


allUniv = epy.getDatabaseFromOSM(
    'universidades', 
    'amenity',
    False,
    True, 
    locationAreaBoundingBox, 
    'university')

In [8]:
#Add universities
epy.mapAddStructureSimpleMarker(map, allUniv, 'glyphicon-home', 'blue', 'name')

In [9]:
distance = 1
barsNear = [data 
     for uni in allUniv for data in allBars 
     if epy.getDistanceInKm(uni["geometry"][0], 
                               uni["geometry"][1],
                               data["geometry"][0], 
                               data["geometry"][1]) < distance]
restaurants = [data 
     for uni in allUniv for data in allRestaurants 
     if epy.getDistanceInKm(uni["geometry"][0], 
                               uni["geometry"][1],
                               data["geometry"][0], 
                               data["geometry"][1]) < distance]

fastFoodBars = [data 
     for uni in allUniv for data in allFastFood 
     if epy.getDistanceInKm(uni["geometry"][0], 
                               uni["geometry"][1],
                               data["geometry"][0], 
                               data["geometry"][1]) < distance]

In [10]:
epy.mapAddStructureSimpleMarker(map, barsNear, 'glyphicon-glass', 'black', 'name')
epy.mapAddStructureSimpleMarker(map, restaurants, 'glyphicon-cutlery', 'green', 'name')
epy.mapAddStructureSimpleMarker(map, fastFoodBars, 'glyphicon-asterisk', 'red', 'name')


In [11]:
epy.mapSave(map, '../reports/maps/universistyAndGastronomy.html')